# Download

In [1]:
import pandas as pd
import regex as re
import requests
import math
import time
import json
import datetime
import os
from dateutil import parser
from datetime import datetime, timedelta
from tqdm.autonotebook import tqdm

/Users/max/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
url_user_pass = {}
with open('url_user_pass.json') as json_file:
    url_user_pass = json.load(json_file)
    url_user_pass = url_user_pass.get("DownloadLogs")

In [3]:
# print(url_user_pass.get("suser"))

In [4]:
session = requests.Session()
session.auth = (url_user_pass.get("Qsuser"), url_user_pass.get("Qpassword"))
# session.auth = (url_user_pass.get("Qsuser"), url_user_pass.get("Qpassword"))

In [7]:
url = url_user_pass.get("other").get("qasurl")
# url = url_user_pass.get("other").get("url")
# url = url_user_pass.get("other").get("url2")
res = session.get(str(url))
contents = res.content.decode('utf8')
# print(contents)
data = json.loads(contents)
# print(data)

In [8]:
# dict(data)['d']['results']

In [2]:
technicalnames = []
for i in data.get('d').get('results'):
    technicalnames.append(i.get('TechnicalName'))
print(technicalnames)

In [1]:
artifacts = []
artpack = {}
pbar = tqdm(total=len(technicalnames))
for i in technicalnames:
    url = str(url_user_pass.get("other").get("qaspartialurl_content"))+"('"+i+"')/Artifacts?&$format=json"
    res = session.get(url)
    contents = res.content.decode('utf8')
    # print(contents)
    data = json.loads(contents)
    artifactlist = []
    for j in data.get('d').get('results'):
        artifactlist.append(j.get('Name'))
#         print(j.get('Name'))
        artifacts.append(j.get('Name'))
    artpack.update({i:artifactlist})
    pbar.update(1)
pbar.close()
print(artpack)

In [206]:
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split,StratifiedKFold,KFold

In [79]:
keyvalpair = []
for i in artpack.keys():
    for j in artpack.get(i):
        keyvalpair.append({'artifact':i, 'iFlow':j})

In [94]:
# df = pd.DataFrame(keyvalpair)
df.describe()

,artifact,iFlow
count,522,522
unique,42,522
top,SFtoBW,OMS1000_RequestToken_SalesforceMarketingCloud
freq,42,1


In [98]:
ratiodfcopy= pd.read_csv("../generateddata/ratiooccuranceclusters.csv")
dfcopy= pd.read_csv("../generateddata/binaryoccuranceclusters.csv")

In [3]:
ratiodfcopy = dropcols(ratiodfcopy)
ratiodfcopy.head()

In [248]:
ratiodfcopy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 2 columns):
scenario    455 non-null object
cluster     455 non-null int64
dtypes: int64(1), object(1)
memory usage: 7.2+ KB


In [4]:
dfcopy = dropcols(dfcopy)
dfcopy.head()

In [112]:
cols = [dfcopy.columns[1],dfcopy.columns[-1]]
def dropcols(df):
    for i in df.columns:
        if i in cols:
            pass
        else:
            df.drop(columns=[i], inplace=True)
    return df

In [93]:
indexNames = df[ (df['artifact'] == 'ASCX12Version005010') | (df['artifact'] == 'QCPIMonitoringDashboard')].index
print(indexNames)
df.drop(indexNames, inplace=True)

Int64Index([], dtype='int64')


In [180]:
X = ratiodfcopy['scenario'].values
y = ratiodfcopy['cluster'].values

In [211]:
# sss = StratifiedShuffleSplit(n_splits=10, train_size=0.154, random_state=42)

# listoflists = []

# for train_index, test_index in sss.split(X, y):
#     print("TRAIN:", train_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     print(X[train_index])
#     listoflists.append(X[train_index])
#     print(len(y[train_index]))
#     for i in X[train_index]:
#         print(i)
#     print("\n")
    
    
# kf = KFold(n_splits=10)
# for train, test in kf.split(X):
# #     print("%s %s" % (train, test))
#     print(X[train_index])
#     listoflists.append(X[train_index])

# skf = KFold(n_splits=10, shuffle=True)
# # skf.get_n_splits(X, y)

# # print(skf)

# for train_index, test_index in skf.split(X, y):
# #     print("TRAIN:", train_index, "TEST:", test_index)
#     listoflists.append(X[train_index])
# #     print(len(X[train_index]))
# #     for i in X[train_index]:
# #         print(i)
# #     print("\n")
# #     X_train, X_test = X[train_index], X[test_index]
# #     y_train, y_test = y[train_index], y[test_index]

# split begin

In [274]:
collegues = ['Brian', 'Matthijs', 'Jennifer', 'Martin', 'Patrick', 'Tom', 'Sven', 'Kenny', 'Peter', 'Rik']
import random
random.shuffle(collegues)
print(collegues)

['Jennifer', 'Matthijs', 'Martin', 'Peter', 'Sven', 'Patrick', 'Rik', 'Kenny', 'Tom', 'Brian']


In [275]:
import numpy as np
random.shuffle(X)
listoflists = np.array_split(X, len(collegues))
for i in listoflists:
    print(len(i))

46
46
46
46
46
45
45
45
45
45


In [276]:
blacklist = []
newlistoflist = []
random.shuffle(X)
for i in listoflists:
    newlist = i.tolist()
    count = 0
    for j in X:
        if len(newlist) == 91:
            break
        if j not in blacklist:
            if j not in i:
                newlist.append(j)
                blacklist.append(j)
                count+=1
    newlistoflist.append(newlist)

ney
ney
ney
ney
ney
ney
aye
aye
ney
ney


In [278]:
from itertools import combinations 
def rSubset(arr, r): 
  
    # return list of all subsets of length r 
    # to deal with duplicate subsets use  
    # set(list(combinations(arr, r))) 
    return list(combinations(arr, r)) 
arr = list(range(0,len(newlistoflist)))
r = 2

valuelist = {}
for w in X:
    valuelist.update({w:1})
# setlist = rSubset(arr, r)
for s in setlist:
    i = s[0]
    j = s[1]
# for i in range(0,len(listoflists)):
    eqsomi = 0
#     for j in range(0,len(listoflists)):
    eqcount = 0
    for x in newlistoflist[i]:
        if x in newlistoflist[j]:
            v = valuelist.get(x,1)
            valuelist.update({x:v+1})
            eqcount+=1
            eqsomi+=1
print(i,eqsomi)
eqsomi = 0
#     print("list {} and {} have {} equalities".format(i,j,eqcount))

8 87


In [291]:
counter = 0
for i in newlistoflist:
    if "NVR2400b_SortingData_EDS_to_BW" in i:
        print(collegues[counter])
    else:
        print("ney")
    df = pd.DataFrame(i,columns=["iFlow"])
    df.to_excel("iFlow rating {}.xlsx".format(collegues[counter]))
    counter+=1

Jennifer
Matthijs
ney
ney
ney
ney
ney
ney
ney
ney


# split eind

In [281]:
# print("{} artifacts".format(len(artpack.keys())))
idart = 0
som = 0
for i in artpack.keys():
    s = len(artpack.get(i))
#     print("Artifact{} & {} \\\\\\hline".format(idart,s))
#     if idart == 26:
#         print(i)
    som += s
    idart+=1
# print(som)

In [13]:
# session = requests.Session()
#
pbar = tqdm(total=len(artifacts))
for i in artifacts:
    url = str(url_user_pass.get("other").get("qaspartialurl_artifacts"))+"(Id='"+i+"',Version='active')/$value"
    res = session.get(url)
#     print(res)
    contents = res.content
    if not str.__contains__(str(contents),"<error"):
        with open('../../../Downloads/PostNLNVR/'+i+'.zip', 'wb') as f:
            f.write(contents)
    pbar.update(1)
pbar.close()
        

In [14]:
import zipfile
import shutil

path = '../../../Downloads/PostNLNVR/'

for i in artifacts:
#     print(i+'.zip')
    if os.path.exists(path+i):
        shutil.rmtree(path+i)
    try:
        with zipfile.ZipFile(path+i+'.zip', 'r') as zip_ref:
            zip_ref.extractall(path+i)
            os.remove(path+i+'.zip')
    except:
        print("Nope: " + i+'.zip')

In [31]:

# import glob
# ziplist = []
# for root, dirs, files in os.walk("../../../Downloads/PostNLQAS"):
#     ziplist = ziplist + glob.glob(root+'/*.zip')

In [37]:
# for i in ziplist:
#     try:
#         with zipfile.ZipFile(i, 'r') as zip_ref:
#                 zip_ref.extractall(i.split("/")[-1].split(".zip")[0])
#                 os.remove(i)
#     except:
#         print("Nope: " + i)

# GIT

In [56]:
import git
from git import Repo

In [57]:
repo = Repo(".")
assert not repo.bare

In [58]:
repo.config_reader()             # get a config reader for read-only access
with repo.config_writer():       # get a config writer to change configuration
    pass     

In [68]:
repo.untracked_files             # retrieve a list of untracked files

[]

In [69]:
repo.is_dirty()

True

In [65]:
assert os.path.isdir(repo.working_tree_dir)                   # directory with your work files
assert repo.git_dir.startswith(repo.working_tree_dir)  # directory containing the git repository

In [123]:
hcommit = repo.head.commit
hcommit.diff(None) 

In [126]:
no_actual_changes = []
changes = []

for diff_added in hcommit.diff(None, create_patch=True):
    if str.__contains__(diff_added.a_blob.path,"parameters.prop"):
        m = re.match(r"@@ \-[\d,]*? \+[\d,]*? @@\n.*?\d{2} \d{2}:\d{2}:\d{2} UTC \d{4}\n.*?\d{2} \d{2}:\d{2}:\d{2} UTC \d{4}([\!-\ÿ\n ]*)", diff_added.diff.decode("utf-8"))
        if m == None:
            changes.append(diff_added.a_blob.path)
        else:
#             no_actual_changes.append(diff_added.a_blob.path)
            if not str.__contains__(m.group(1), "+"):
                no_actual_changes.append(diff_added.a_blob.path)
            else:
                changes.append(diff_added.a_blob.path)
    else:
        changes.append(diff_added.a_blob.path)


In [131]:
index = repo.index
index.add(changes)                                             # add a new file to the index

repo.active_branch.commit = repo.commit('HEAD~1')                      # forget last commit

from git import Actor
author = Actor("PythonClient", "max.nijholt@qforit.com")
committer = Actor("Max Nijholt", "max.nijholt@qforit.com")
# commit by commit message and author and committer
# index.commit("Update", author=author, committer=committer)

<git.Commit "dd73a817d96b9f1e87e00ffb19b3ab5e4e36cfc6">

In [132]:
git = repo.git

In [136]:
PATH_OF_GIT_REPO = repo.working_tree_dir + "\.git"  # make sure .git folder is properly configured
COMMIT_MESSAGE = 'Update'

def git_push():
    try:
        repo = Repo(PATH_OF_GIT_REPO)
        repo.git.add(update=True)
        repo.index.commit(COMMIT_MESSAGE)
        origin = repo.remote(name='origin')
        origin.push()
    except:
        print('Some error occured while pushing the code')    

git_push()


Some error occured while pushing the code
